In [1]:
import pandas as pd
import os
import glob
import numpy as np
import smartsheet

#Variables
# random
excel = '.xlsx'
csv = '.csv'

# smartsheets Shet IDs
Central_id = '1844440261257092'
South_id= '6292262490531716'

# Central's API
Central = smartsheet.Smartsheet('6jJFxnAfFsDErU0BSr1Ae4D6cHq0zenczVR12') #API

Central.errors_as_exceptions(True)

# Central's API
South = smartsheet.Smartsheet("H8mqAo60J2zGHSrJEwmlMoY88A9yjgCo6RnSI") # API
South.errors_as_exceptions(True)

#download smartsheets to any folder you need
# Central.Sheets.get_sheet_as_excel(Central_id, download_path=r"C:\Users\jacque.trahan\Downloads")
# South.Sheets.get_sheet_as_excel(South_id, download_path=r"C:\Users\jacque.trahan\Downloads") 


#Google Sheets
#Sheet ID comes right after /d/ in the URL
Google_sheet_ID= '1O2dmdhEo5hJVAgRVDl4M4n3_v-KqHS56q3OZVdpZANU'
# name of tab you want to use
Google_sheets_name = 'Tree RTR Tracker'

# this is tricky. after /export? we must put format=csv#id={Sheet Name}   -> manually to make this work
url = f'https://docs.google.com/spreadsheets/d/{Google_sheet_ID}/export?format=csv#id={Google_sheets_name}'

# Function to read in column titles and row vales and make out data frames
def smartsheet_loadup(sheet):
    columns = [col.title for col in sheet.columns]
    rows = []
    for row in sheet.rows:
        cells = []
        for cell in row.cells:
            cells.append(cell.value)
        rows.append(cells)
    df = pd.DataFrame(rows, columns=columns)
    return df

Central_SS = smartsheet_loadup(Central.Sheets.get_sheet(Central_id))
South_SS = smartsheet_loadup(South.Sheets.get_sheet(South_id))

df_both = pd.concat([Central_SS,South_SS])

In [2]:
#states DTSC tracker
path = r"C:\Users\jacque.trahan\Downloads\Fires2021_ParcelLayer_v2_InternalView*"
file_type = '.xlsx'
loadup = glob.glob(path + file_type)
df = pd.read_excel(max(loadup, key=os.path.getctime), sheet_name=1)

In [3]:
# df[df['County'].isin(['NEVADA'])].to_excel('')

# Navada

In [4]:
navada = df[(df['County'].isin(['NEVADA'])) & (df['Parcel Status:'].isin(['Assessed', 'Deferred to Phase 2',
                                                                             'Identified', 'Assessed Return Needed']))]

In [5]:
navada = navada.copy()

In [6]:
navada['APN'] = navada['APN'].astype(str)

In [7]:
navada['APN'] = ( "0" + navada['APN'].str[:-3])

In [8]:
# navada['APN'] 

# placer

In [9]:
placer = df[(df['County'].isin(['PLACER'])) & (df['Parcel Status:'].isin(['Assessed', 'Deferred to Phase 2',
                                                                             'Identified', 'Assessed Return Needed']))]

In [10]:
placer =placer.copy()

In [11]:
placer['APN'] = placer['APN'].astype(str)

In [12]:
placer['APN'] = placer['APN'].str.replace("-", "")

In [13]:
placer['APN'] = placer['APN'].str[:-3]

In [14]:
# placer['APN']

# Lake

In [15]:
lake = df[(df['County'].isin(['LAKE'])) & (df['Parcel Status:'].isin(['Assessed', 'Deferred to Phase 2',
                                                                             'Identified', 'Assessed Return Needed']))]

In [16]:
lake = lake.copy()

In [17]:
lake['APN'] = lake['APN'].astype(str)

In [18]:
lake['APN'] = ("0" + lake['APN'] + "0")

In [19]:
# lake['APN']

# EL DORADO

In [20]:
eldorado = df[(df['County'].isin(['EL DORADO'])) & (df['Parcel Status:'].isin(['Assessed', 'Deferred to Phase 2',
                                                                             'Identified', 'Assessed Return Needed']))]

In [21]:
eldorado = eldorado.copy()

In [22]:
eldorado['APN'] = eldorado['APN'].astype(str)

In [23]:
eldorado['APN'] = "0" + eldorado['APN']

# Kern

* no APNs in smartsheets yet.
* will need to reformat APNs

In [24]:
kern = df[(df['County'].isin(['KERN'])) & (df['Parcel Status:'].isin(['Assessed', 'Deferred to Phase 2',
                                                                             'Identified', 'Assessed Return Needed']))]
kern = kern.copy()
kern['APN'] = kern['APN'].astype(str)


kern['APN'] = "0" + kern['APN']

# Mendocino

In [25]:
mendo = df[(df['County'].isin(['MENDOCINO'])) & (df['Parcel Status:'].isin(['Assessed', 'Deferred to Phase 2',
                                                                             'Identified', 'Assessed Return Needed']))]
mendo = mendo.copy()
mendo['APN'] = mendo['APN'].astype(str)

mendo['APN'] = (mendo['APN'] + "0")

# Tulare

* no APNs in smartsheets yet.
* will need to reformat APNs

In [26]:
tulare = df[(df['County'].isin(['TULARE'])) & (df['Parcel Status:'].isin(['Assessed', 'Deferred to Phase 2',
                                                                             'Identified', 'Assessed Return Needed']))] 
tulare = tulare.copy()
tulare['APN'] = tulare['APN'].astype(str)
tulare['APN'] = tulare['APN'].str.replace("-","")

# Tuolumne

In [27]:
tulumne = df[(df['County'].isin(['TUOLUMNE'])) & (df['Parcel Status:'].isin(['Assessed', 'Deferred to Phase 2',
                                                                             'Identified', 'Assessed Return Needed']))] 
tulumne = tulumne.copy()
tulumne['APN'] = tulumne['APN'].astype(str)
tulumne['APN'] = tulumne['APN'].str.replace("-","")

In [28]:
county = [navada,placer,lake, eldorado,mendo,tulumne,tulare,kern]

# concat all the counties into one file

In [29]:
DTSC = pd.concat(county)

### Add the dashes

In [30]:
DTSC['APN'] = (DTSC['APN'].str[:3] + '-' + DTSC['APN'].str[3:6] + '-' + DTSC['APN'].str[6:])

In [31]:
DTSC = DTSC[['APN', 'County', 'Parcel Status:', 'Address']]

In [32]:
# DTSC[DTSC['County'].isin(['MENDOCINO', 'TUOLUMNE'])]

In [33]:
DTSC_copy = DTSC.copy()
DTSC_copy = DTSC_copy.add_suffix('_DTSC')

# For Randy - Needs Site Assessment Filter

In [34]:
# path3 = r"C:\Users\jacque.trahan\Downloads\Randy filter*"
# file_type3 = '.xlsx'
# load_it3 = glob.glob(path3 + file_type3)
# df3 = pd.read_excel(max(load_it3, key=os.path.getctime))

In [35]:
# Randy_filt = df3.copy()

In [36]:
# Randy_filt = Randy_filt[['APN_ROW Segment', 'ROE Verified','ROE Type', 'Hold',
#                          'Not Eligible','Withdrawal','Phase 1 Complete','Site Assessment', 'USFS Parcel?']]

In [37]:
# Randy_filt = Randy_filt.add_suffix('_Randy')

In [38]:
# DTSC_Randy = DTSC_copy.merge(Randy_filt,
#                                            left_on='APN_DTSC',
#                                            right_on='APN_ROW Segment_Randy',
#                                            how='outer')

# Planning Ready for site assessment tracker. 

## filters for ROEs Available for SA Teams
* ROE Verified = True
* Holds = False
* Not eilibale and widrawn = blank
* phase one = blank
* Site Assessmet = blank
* USFS Parcel? = N/A

## Smartsheet now ran through API and no longer needed to be downloaded

In [39]:
# #sSmart sheets central
# path2 = r"C:\Users\jacque.trahan\Downloads\Central Division 2021 Debris Removal Tracker*"
# file_type2 = '.xlsx'
# loadup2 = glob.glob(path2 + file_type2)
# df2 = pd.read_excel(max(loadup2, key=os.path.getctime))# for ready SA properties
# df_ss = pd.read_excel(max(loadup2, key=os.path.getctime), keep_default_na=False)

In [40]:
# path3 = r"C:\Users\jacque.trahan\Downloads\South Central Division 2021 Debris Removal Tracker*"
# file_type2 = '.xlsx'
# loadup3 = glob.glob(path3 + file_type2)
# df2_s = pd.read_excel(max(loadup3, key=os.path.getctime))# for ready SA properties
# df_ss_s = pd.read_excel(max(loadup3, key=os.path.getctime), keep_default_na=False)

# concate files for both sets of smartsheets

In [41]:
# df_ss_both = pd.concat([df_ss, df_ss_s])

# df2_both = pd.concat([df2, df2_s])

In [42]:
SmartSheets = df_both.copy()
SmartSheets = SmartSheets[['APN_ROW Segment','Street #','Street Name','County',
                           'ROE Verified','ROE Type', 'Hold', 'Hold Reason',
                         'Not Eligible','Withdrawal','Phase 1 Complete','Site Assessment', 'USFS Parcel?', 'Debris Crew']]

In [43]:
rdy_SAs = df_both.copy()
rdy_SAs = rdy_SAs[(rdy_SAs['ROE Verified'] == 'ROE Verified') & # Roe verified
           (rdy_SAs['ROE Type'] == 'Full Program ROE (debris + trees)') & # ROE types is not just trees
           (rdy_SAs['Hold'] != True) & # not on hold 
           (rdy_SAs['Not Eligible'].isnull()) & # Not eilibale = blank
           (rdy_SAs['Withdrawal'].isnull()) & # Withdrawal= blank
           (rdy_SAs['Phase 1 Complete'] == 'Yes') & #Phase one complete
           (rdy_SAs['Site Assessment'].isna()) #& # SA hasnt been done
#            (rdy_SAs['USFS Parcel?'].notna())           
           ]

In [44]:
rdy_SAs = rdy_SAs[['APN_ROW Segment','Street #','Street Name','County',
                   'ROE Verified','ROE Type', 'Hold',
                         'Not Eligible','Withdrawal','Phase 1 Complete','Site Assessment', 'USFS Parcel?']]

In [45]:
# rdy_SAs[(rdy_SAs['Not Eligible'].isnull())]

In [46]:
rdy_SAs = rdy_SAs.add_suffix("_Planning")

In [47]:
SS_merge = rdy_SAs.merge(SmartSheets,
                            left_on='APN_ROW Segment_Planning',
                            right_on='APN_ROW Segment',
                            how='outer')

In [48]:
# SS_merge_RandyFilt = SS_merge.merge(Randy_filt,
#                                left_on='APN_ROW Segment',
#                                right_on='APN_ROW Segment_Randy',
#                                how='outer')

In [49]:
final = SS_merge.merge(DTSC_copy,
                                 left_on='APN_ROW Segment',
                                 right_on='APN_DTSC',
                                 how='outer')

In [50]:
final.set_index('APN_ROW Segment', inplace=True)

In [51]:
final.to_excel('DTSC and rdy for SA check - Central.xlsx')